In [1]:
import os
import json
import pickle
from torchvision.datasets import CocoDetection
from pycocotools.coco import COCO

In [2]:
with open("/home/atuin/g103ea/shared/coco2014/coco_split.json", "r") as f:
    splits = json.load(f)

In [3]:
valid_image_ids = []
for split in ["test"]:
    valid_image_ids.extend(splits[split])

In [4]:
dataset = CocoDetection(
    "/home/atuin/g103ea/shared/coco2014/images",
    "/home/atuin/g103ea/shared/coco2014/annotations/instances_all2014.json",
)

loading annotations into memory...


Done (t=16.60s)
creating index...
index created!


In [5]:
label_ids = {}
for i, j in dataset.coco.cats.items():
    label_ids[j["name"]] = i

In [37]:
target_categories = [
    "bottle"
]
coco_captions = COCO(
    "/home/atuin/g103ea/shared/coco2014/annotations/captions_all2014.json"
)
get_captions = lambda image_id: [
    ann["caption"] for ann in coco_captions.imgToAnns[image_id]
]

loading annotations into memory...


Done (t=1.75s)
creating index...
index created!


In [38]:
for cat in target_categories:
    valid_images = []
    for img_id in dataset.coco.catToImgs[label_ids[cat]]:
        if img_id in valid_image_ids:
            valid_images.append(
                {
                    "image_id": img_id,
                    "image_path": os.path.join(
                        "/home/atuin/g103ea/shared/coco2014/images",
                        dataset.coco.imgs[img_id]["file_name"],
                    ),
                    "captions": get_captions(img_id),
                }
            )
    print(cat, "--->", len(valid_images))
    pickle.dump(
        valid_images,
        open(
            os.path.join(
                "/home/hpc/g103ea/g103ea14/mai/CLIP_prefix_caption/test_data",
                cat + ".pkl",
            ),
            "wb",
        ),
    )

bottle ---> 1128


In [15]:
import pickle
import numpy as np
import os
results = []
images_dir = "/home/hpc/g103ea/g103ea14/mai/CLIP_prefix_caption/test_data"
dest_dir = "/home/hpc/g103ea/g103ea14/mai/CLIP_prefix_caption/test_subset"
for cat1, cat2 in cc.items():
    images = pickle.load(open(os.path.join(images_dir, cat1+".pkl"), "rb"))
    images_d = {i["image_id"]: i for i in images}
    print(len(images_d))

    valid_ids = filter_image_ids(sorted(images_d.keys()), label_ids[cat2])
    valid_ids = np.random.choice(valid_ids, min(1000, len(valid_ids)), replace=False)
    print(len(valid_ids))

    valid_images = [images_d[i] for i in valid_ids]
    print(len(valid_images))
    
    pickle.dump(valid_images, open(os.path.join(dest_dir, cat1+".pkl"), "wb"))
    print(f"Added {len(valid_images)} / {len(images_d)} valid images for {cat1}, {valid_images[0]}")
    print(f"added to path {os.path.join(dest_dir, cat1+'.pkl')}")
    print("-----------------------------------------------------")

143
141
141
Added 141 / 143 valid images for skateboard, {'image_id': 274455, 'image_path': '/home/atuin/g103ea/shared/coco2014/images/COCO_val2014_000000274455.jpg', 'captions': ['A man riding a skateboard on top of pavement.', 'A skateboarder is taking a break at a skatepark. ', 'A man at a skate park with his foot on the side of the skateboard.', "A person with their food on a skateboard that's on the ground.", 'A skateboarder with his foot on his board that is laying on its side.']}
added to path /home/hpc/g103ea/g103ea14/mai/CLIP_prefix_caption/test_subset/skateboard.pkl
-----------------------------------------------------
118
115
115
Added 115 / 118 valid images for horse, {'image_id': 449274, 'image_path': '/home/atuin/g103ea/shared/coco2014/images/COCO_val2014_000000449274.jpg', 'captions': ['A horse running on a dirt road in the open.', 'The rider view of a horse ride on the beach.', 'there is a horse that is walking along the water on the beach', 'A horse is walking in the s